# Import, Color Palette, and Functions

In [1]:
pwd

'/Users/davidl/modules/cpr/agotool'

In [2]:
cd app/python

/Users/davidl/modules/cpr/agotool/app/python


In [16]:
import os, sys
homedir = os.path.expanduser("~")
import create_SQL_tables_snakemake as cst
import obo_parser
from importlib import reload
reload(cst)

import numpy as np
import pandas as pd
import matplotlib
%matplotlib inline

import bokeh.sampledata
# bokeh.sampledata.download()
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool, BoxZoomTool, ResetTool, LassoSelectTool, WheelZoomTool, PanTool, SaveTool, TapTool, Legend
from bokeh.io import output_notebook, export_png, show
output_notebook()

from bokeh.palettes import Spectral
from bokeh import palettes

Loading BokehJS ...

In [8]:
# https://github.com/groundhogday321/python-bokeh/blob/master/Python%20Bokeh.ipynb
# configure the default output state to generate output in notebook cells when show() is called
# in colab, output_notebook() is called in each cell (this is not always the case)
# output_notebook()
# output_file()
# comment out output_file(), run reset_output(), then run output_notebook() to keep from opening new tabs

# Save file
# save(p)
# Print out div and script
# script, div = components(p)
# print(div)
# print(script)


# from https://docs.bokeh.org/en/latest/docs/reference/palettes.html
# and http://colorbrewer2.org/#type=qualitative&scheme=Dark2&n=4
# Category20 from 3 - 20 but use only for 13 and 14
# or Category10 from 3 - 10 

palette_dict = {}
palette_dict[1] = ['#d95f02']
palette_dict[2] = ['#1b9e77','#d95f02']
palette_dict[3] = palettes.d3['Category10'][3] # ['#1b9e77','#d95f02','#7570b3']
palette_dict[4] = palettes.d3['Category10'][4] # ['#e41a1c','#377eb8','#4daf4a','#984ea3'] # ['#1b9e77','#d95f02','#7570b3','#e7298a']
palette_dict[5] = palettes.d3['Category10'][5] # ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e']
palette_dict[6] = palettes.d3['Category10'][6] # ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02']
palette_dict[7] = palettes.d3['Category10'][7] # ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d']
palette_dict[8] = palettes.d3['Category10'][8] # ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d','#666666']
palette_dict[9] = palettes.d3['Category10'][9] # ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#ffff33','#a65628','#f781bf','#999999']
palette_dict[10] = palettes.d3['Category10'][10] # ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a']
palette_dict[11] = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99']
palette_dict[12] = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99','#b15928']
palette_dict[13] = ['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5', '#8c564b', '#c49c94', '#e377c2'] # palettes.d3['Category20'][13]
palette_dict[14] = ['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5', '#8c564b', '#c49c94', '#e377c2', '#f7b6d2'] # palettes.d3['Category20'][14]

import query, variables
import math
from collections import namedtuple

Edge = namedtuple("Edge", ["x_vals", "y_vals", "weight"])

def scale_FG_count_2_plot_circle_size(df, min_circle_size=6, max_circle_size=22):
    """
    used to scale size of circles
    highest values of FG_count gets scaled down to max_circle_size
    lowest value or values below min_circle_size are set to min_circle_size
    """
    max_FG_count = max(df["FG_count"])
    scaling_factor_circle_size = max_circle_size/max_FG_count
    FG_count_2_circle_size = df["FG_count"].apply(lambda x: x * scaling_factor_circle_size)
    FG_count_2_circle_size[FG_count_2_circle_size < min_circle_size] = min_circle_size
    return FG_count_2_circle_size    

def scale_rank_2_plot_transparency(rank, min_fill_alpha=0.5):
    """    
    used for filling alpha (transparency)
    # 1 = 1
    # 2 = 0.9
    # 3 = 0.8 
    # 4 = 0.7
    # 5 = 0.6
    # 6 = 0.5
    # 7 = 0.5
    # 999 = 0.5
    # rank = 1
    # 1 - (rank - 1)/10
    """
    scaled = 1 - (rank - 1)/10
    if scaled < min_fill_alpha:
        return min_fill_alpha
    else:
        return scaled
    
def add_color_per_etype(df, palette_dict):
    etypes_unique = df["etype"].unique()
    num_colors = etypes_unique.shape[0]
    try:
        color_palette = palette_dict[num_colors]
    except KeyError:
        print("There are {} number of colors to be set, but palette_dict only has {} unique palettes.".format(num_colors, palette_dict))
        raise StopIteration
    df["color"] = ""
    count = 0
    for etype in sorted(etypes_unique)[::-1]:
        cond = df["etype"] == etype
        df.loc[cond, "color"] = color_palette[count]
        count += 1
    return df["color"]

from itertools import combinations

def yield_element_plus_rest_of_list(list_):
    for i in range(len(list_)):
        yield list_[i], list_[:i] + list_[i+1:]

def get_edge_positions_and_weight(df):
    # for each pair of row-combinations, get x and y, start and stop positions, plus weight (Jaccard index)
    edges_plus_weight_list = [] # [(x_start_2_stop, y_start_2_stop, weight), ...]
    for first_index, second_index in combinations(df.index, 2):
        try:
            first_FG_IDs = set(df.loc[first_index, "FG_IDs"].split(";"))
            second_FG_IDs = set(df.loc[second_index, "FG_IDs"].split(";"))
            # compute weight of edge
            jaccard_index = len(first_FG_IDs.intersection(second_FG_IDs)) / len(first_FG_IDs.union(second_FG_IDs))
        except: #  AttributeError if NaN or ZeroDivisionError if ... 
            jaccard_index = 1
        x_start_2_stop = (df.loc[first_index, "logFDR"], df.loc[second_index, "logFDR"])
        y_start_2_stop = (df.loc[first_index, "effectSize"], df.loc[second_index, "effectSize"])        
        edges_plus_weight_list.append(Edge(x_start_2_stop, y_start_2_stop, jaccard_index))
    return edges_plus_weight_list

def find_edges_via_hierarchy(df, lineage_dict_enum, v1=False):
    if v1:
        return find_edges_via_hierarchy_v1(df, lineage_dict_enum)
    else:
        return find_edges_via_hierarchy_v2(df, lineage_dict_enum)

def find_edges_via_hierarchy_v1(df, lineage_dict_enum):
    """
    for each etype find edges to draw in plot of x="logFDR", y="effectSize"
    add funcEnum and lineage to set
    v1: merge sets if there is overlapp --> wrong since everything get connected
    v2: don't merge set, simply check if >= 2 funcEnums from given user data present
    filter by funcEnums that are in the given user data
    e.g. of return
    {-57: [],
     -51: [{47939, 47977, 48117, 48335, 48487}],
     ...}     
    df: DataFrame of aGOtool output with ['funcEnum', 'etype'] cols
    lineage_dict_enum: key: function enumeration, value: set of func enum array all parents
    :return: etype_2_funcEnumFamilySet_list_dict, key: etype(int), val: list of sets (int)
    """
    etype_2_funcEnumFamilySet_dict, etype_2_edgesXYCoords_list_dict = {}, {} 
    for etype, group in df.groupby("etype"):
        funcEnumFamilySet_list_temp, funcEnumPair_set, edgesXYCoords_list = [], set(), [] #funcEnumFamilySet_list_merged
        funcEnumFamilySet_list_merged = []
        if etype not in variables.entity_types_with_ontology:
            continue
        funcEnums_2_travers_set = set(group["funcEnum"])
        for funcEnum in funcEnums_2_travers_set:
            lineage = lineage_dict_enum.get(funcEnum, False)
            if not lineage: 
                continue
            lineage_2_add = set(lineage)
            lineage_2_add.add(funcEnum)            
            funcEnumFamilySet_list_temp.append(lineage_2_add)
            
        for funcEnumFamilySet, funcEnumFamilySet_list_2_merge in yield_element_plus_rest_of_list(funcEnumFamilySet_list_temp):
            for set_2_merge in funcEnumFamilySet_list_2_merge:
                if len(funcEnumFamilySet.intersection(set_2_merge)) > 0:
                    funcEnumFamilySet = funcEnumFamilySet.union(set_2_merge)
            funcEnumFamilySet = funcEnumFamilySet.intersection(funcEnums_2_travers_set)   
            if funcEnumFamilySet not in funcEnumFamilySet_list_merged and len(funcEnumFamilySet) > 1:
                funcEnumFamilySet_list_merged.append(funcEnumFamilySet)
                # extract x and y coordinates and a weight depending on the overlapp of FG_IDs (Jaccard index)
                edges_plus_weight_list = get_edge_positions_and_weight(group[group["funcEnum"].isin(funcEnumFamilySet)])                
                edgesXYCoords_list += edges_plus_weight_list
                                
        etype_2_funcEnumFamilySet_dict[etype] = funcEnumFamilySet_list_merged
        etype_2_edgesXYCoords_list_dict[etype] = edgesXYCoords_list
        
    return etype_2_funcEnumFamilySet_dict, etype_2_edgesXYCoords_list_dict

def find_edges_via_hierarchy_v2(df, lineage_dict_enum):
    """
    for each etype find edges to draw in plot of x="logFDR", y="effectSize"
    add funcEnum and lineage to set
    v1: merge sets if there is overlapp --> wrong since everything get connected
    v2: don't merge set, simply check if >= 2 funcEnums from given user data present
    filter by funcEnums that are in the given user data
    e.g. of return
    {-57: [],
     -51: [{47939, 47977, 48117, 48335, 48487}],
     ...}     
    df: DataFrame of aGOtool output with ['funcEnum', 'etype'] cols
    lineage_dict_enum: key: function enumeration, value: set of func enum array all parents
    :return: etype_2_funcEnumFamilySet_list_dict, key: etype(int), val: list of sets (int)
    """
    etype_2_funcEnumPair_set_dict, etype_2_edgesXYCoords_list_dict = {}, {} 
    for etype, group in df.groupby("etype"):
        funcEnumFamilySet_list_temp, funcEnumPair_set, edgesXYCoords_list = [], set(), [] #funcEnumFamilySet_list_merged
        funcEnumFamilySet_list_merged = []
        if etype not in variables.entity_types_with_ontology:
            continue
        funcEnums_2_travers_set = set(group["funcEnum"])
        for funcEnum in funcEnums_2_travers_set:
            lineage = lineage_dict_enum.get(funcEnum, False)
            if not lineage: 
                continue
            lineage_2_add = set(lineage)
            lineage_2_add.add(funcEnum)                    
            funcEnumFamilySet = lineage_2_add.intersection(funcEnums_2_travers_set)
            len_funcEnumFamilySet = len(funcEnumFamilySet)
            if len_funcEnumFamilySet < 2:
                continue
            elif len_funcEnumFamilySet == 2:
                funcEnumPair_set.add(tuple(sorted(funcEnumFamilySet)))
            elif len_funcEnumFamilySet > 2:
                for pair in combinations(funcEnumFamilySet, 2):
                    funcEnumPair_set.add(tuple(sorted(pair)))                
            else:
                raise StopIteration
        ## extract x and y coordinates and a weight depending on the overlapp of FG_IDs (Jaccard index)
        for funcEnumPair in funcEnumPair_set:
            edgesXYCoords_list += get_edge_positions_and_weight(group[group["funcEnum"].isin(funcEnumPair)])    
                    
        etype_2_funcEnumPair_set_dict[etype] = funcEnumPair_set #funcEnumFamilySet_list_merged
        etype_2_edgesXYCoords_list_dict[etype] = edgesXYCoords_list
        
    return etype_2_funcEnumPair_set_dict, etype_2_edgesXYCoords_list_dict

def get_edge_renderer(df, lineage_dict_enum):
    """
    edge_renderer 
    dict: {"start": [node_index_start_1, node_index_start_2, ...], "end": [node_index_end_1, node_index_end_2, ...]}
    """
    etype_2_edgeRenderer_dict = {}
    for etype, group in df.groupby("etype"):
        etype_2_edgeRenderer_dict[etype] = {"start": [], "end": []}
        funcEnumFamilySet_list_temp, funcEnumPair_set, edgesXYCoords_list = [], set(), [] #funcEnumFamilySet_list_merged
        funcEnumFamilySet_list_merged = []
        if etype not in variables.entity_types_with_ontology:
            continue
        funcEnums_2_travers_set = set(group["funcEnum"])
        for funcEnum in funcEnums_2_travers_set:
            lineage = lineage_dict_enum.get(funcEnum, False)
            if not lineage: 
                continue
            lineage_2_add = set(lineage)
            lineage_2_add.add(funcEnum)                    
            funcEnumFamilySet = lineage_2_add.intersection(funcEnums_2_travers_set)
            len_funcEnumFamilySet = len(funcEnumFamilySet)
            if len_funcEnumFamilySet < 2:
                continue
            elif len_funcEnumFamilySet == 2:
                funcEnumPair_set.add(tuple(sorted(funcEnumFamilySet)))
            elif len_funcEnumFamilySet > 2:
                for pair in combinations(funcEnumFamilySet, 2):
                    funcEnumPair_set.add(tuple(sorted(pair)))                
            else:
                raise StopIteration
        ## extract x and y coordinates and a weight depending on the overlapp of FG_IDs (Jaccard index)
        for funcEnumPair in funcEnumPair_set:
            node_index_start_node_index_end = group[group["funcEnum"].isin(funcEnumPair)].index.tolist()
            assert len(node_index_start_node_index_end) == 2
            node_index_start, node_index_end = node_index_start_node_index_end
            etype_2_edgeRenderer_dict[etype]["start"].append(node_index_start)
            etype_2_edgeRenderer_dict[etype]["end"].append(node_index_end)
    return etype_2_edgeRenderer_dict

def get_graph_layout(df):
    """
    graph_layout # {node_index: (x_pos, y_pos), ...}
    """
#     return {index_: x_y_pos_tuple for index_, x_y_pos_tuple in zip(df.index.tolist(), list(zip(df["logFDR"], df["effectSize"])))}
    etype_2_graph_layout_dict = {}
    for etype, group in df.groupby("etype"):
        etype_2_graph_layout_dict[etype] = {}
        for index_, x_y_pos_tuple in zip(group.index.tolist(), list(zip(group["logFDR"], group["effectSize"]))):
            etype_2_graph_layout_dict[etype][index_] = x_y_pos_tuple
    return etype_2_graph_layout_dict

def find_edges_via_hierarchy_v3(df, lineage_dict_direct):
    """
    for each etype find edges to draw in plot of x="logFDR", y="effectSize"
    add funcEnum and lineage to set
    v1: merge sets if there is overlapp --> wrong since everything get connected
    v2: don't merge set, simply check if >= 2 funcEnums from given user data present
    filter by funcEnums that are in the given user data
    v3: iterate over direct parents and stop once at least one is found
    e.g. of return
    {-57: [],
     -51: [{47939, 47977, 48117, 48335, 48487}],
     ...}     
    df: DataFrame of aGOtool output with ['funcEnum', 'etype'] cols
    lineage_dict_enum: key: function enumeration, value: set of func enum array all parents
    :return: etype_2_funcEnumFamilySet_list_dict, key: etype(int), val: list of sets (int)
    """
    x_startL, x_stopL, y_startL, y_stopL, weightL, etypeL = [], [], [], [], [], []
    etype_2_funcNamePair_set_dict, etype_2_edgesXYCoords_list_dict = {}, {} 
    for etype, group in df.groupby("etype"):
        funcNameFamilySet_list_merged, funcNameFamilySet_list_temp, edgesXYCoords_list, funcNamePair_set = [], [], [], set()
        if etype not in variables.entity_types_with_ontology:
            continue
        funcs_2_travers_set = group["term"]
        for term in funcs_2_travers_set:
            funcNamePair_set |= find_pairs_2_link(term, lineage_dict_direct, funcs_2_travers_set)
                
        ## extract x and y coordinates and a weight depending on the overlapp of FG_IDs (Jaccard index)
        for funcNamePair in funcNamePair_set:
            t = get_edge_positions_and_weight_as_list(group[group["term"].isin(funcNamePair)])
            x_startL_2merge, x_stopL_2merge, y_startL_2merge, y_stopL_2merge, weightL_2merge = t
            x_startL += x_startL_2merge
            x_stopL += x_stopL_2merge
            y_startL += y_startL_2merge
            y_stopL += y_stopL_2merge
            weightL += weightL_2merge
            etypeL += len(weightL_2merge) * [etype]                                        
        etype_2_funcNamePair_set_dict[etype] = funcNamePair_set
#         etype_2_edgesXYCoords_list_dict[etype] = edgesXYCoords_list    
    df_coords = pd.DataFrame()
    df_coords["etype"] = etypeL
    df_coords["x_start"] = x_startL
    df_coords["x_stop"] = x_stopL    
    df_coords["y_start"] = y_startL
    df_coords["y_stop"] = y_stopL
    df_coords["weight"] = weightL        
    return etype_2_funcNamePair_set_dict, df_coords #etype_2_edgesXYCoords_list_dict

def get_edge_positions_and_weight_as_list(df):
    # for each pair of row-combinations, get x and y, start and stop positions, plus weight (Jaccard index)
#     edges_plus_weight_list = [] # [(x_start_2_stop, y_start_2_stop, weight), ...]
    x_startL, x_stopL, y_startL, y_stopL, weightL = [], [], [], [], []
    for first_index, second_index in combinations(df.index, 2):
        try:
            first_FG_IDs = set(df.loc[first_index, "FG_IDs"].split(";"))
            second_FG_IDs = set(df.loc[second_index, "FG_IDs"].split(";"))
            # compute weight of edge
            jaccard_index = len(first_FG_IDs.intersection(second_FG_IDs)) / len(first_FG_IDs.union(second_FG_IDs))
        except: #  AttributeError if NaN or ZeroDivisionError if ... 
            jaccard_index = 1
#         x_start_2_stop = (df.loc[first_index, "logFDR"], df.loc[second_index, "logFDR"])
#         y_start_2_stop = (df.loc[first_index, "effectSize"], df.loc[second_index, "effectSize"])        
#         edges_plus_weight_list.append(Edge(x_start_2_stop, y_start_2_stop, jaccard_index))
        x_startL.append(df.loc[first_index, "logFDR"])
        x_stopL.append(df.loc[second_index, "logFDR"])
        y_startL.append(df.loc[first_index, "effectSize"])
        y_stopL.append(df.loc[second_index, "effectSize"])
        weightL.append(jaccard_index)
    return x_startL, x_stopL, y_startL, y_stopL, weightL

def find_pairs_2_link(term, lineage_dict_direct, funcs_2_travers_set):
    funcNamePair_set = set()
    for parents in cst.yield_direct_parents([term], lineage_dict_direct): 
        if not parents:
            continue
        else:
            lineage_2_add = set(parents)
            lineage_2_add.add(term)
            funcNameFamilySet = lineage_2_add.intersection(funcs_2_travers_set)
        len_funcNameFamilySet = len(funcNameFamilySet)
        if len_funcNameFamilySet < 2:
            continue
        elif len_funcNameFamilySet == 2:
            funcNamePair_set.add(tuple(sorted(funcNameFamilySet)))
            return funcNamePair_set
        elif len_funcNameFamilySet > 2:
            for func in funcNameFamilySet - set([term]):
                funcNamePair_set.add(tuple(sorted([term, func])))
            return funcNamePair_set
        else:
            raise StopIteration
    return funcNamePair_set


def get_df_ready_for_bokeh_plot(df):
    df = df[(df["FDR"] <= 0.05) & (df["FDR"] > 0)]
    df["logFDR"] = df["FDR"].apply(lambda x: math.log(x)*-1)
    df["FG_count_2_circle_size"] = scale_FG_count_2_plot_circle_size(df)
    df["rank_2_transparency"] = df["rank"].apply(scale_rank_2_plot_transparency)
    df["color"] = add_color_per_etype(df, palette_dict)
    etype_2_funcNamePair_set_dict, etype_2_edgesXYCoords_list_dict = find_edges_via_hierarchy_v3(df, lineage_dict)
    return df, etype_2_edgesXYCoords_list_dict



lineage_dict = cst.get_lineage_dict_for_all_entity_types_with_ontologies(direct_or_allParents="direct")
# etype_2_funcNamePair_set_dict, etype_2_edgesXYCoords_list_dict = find_edges_via_hierarchy_v3(df, lineage_dict)

Loading BokehJS ...

In [9]:
# etype_2_funcNamePair_set_dict, df_coords = find_edges_via_hierarchy_v3(df_nodes, lineage_dict)

entityType_2_functionType_dict = {-20: "Gene Ontology cellular component TEXTMINING",
                              -21: "Gene Ontology biological process",
                              -22: "Gene Ontology cellular component",
                              -23: "Gene Ontology molecular function",
                              -25: "Brenda Tissue Ontology",
                              -26: "Disease Ontology",
                              -51: "UniProt keywords",
                              -52: "KEGG (Kyoto Encyclopedia of Genes and Genomes)",
                              -53: "SMART (Simple Modular Architecture Research Tool)",
                              -54: "INTERPRO",
                              -55: "PFAM (Protein FAMilies)",
                              -56: "PMID (PubMed IDentifier)",
                              -57: "Reactome",
                              -58: "WikiPathways"}

In [18]:
# fn_agotool_output = r"/Users/dblyon/Downloads/results_orig_6763_1582905397.2842093.tsv"
# fn_agotool_output = r"/Users/dblyon/Downloads/results_orig_6764_1582908060.2764244.tsv" # THE example (Yeast, abundance corr, ex1). GOCC is missing
# fn_agotool_output = r"/Users/dblyon/Downloads/results_orig_19683_1588623764.3226633.tsv" 
# fn_agotool_output = r"/Users/dblyon/Downloads/results_orig_22574_1588683317.121278.tsv"
# updated agotool.org
# fn_agotool_output = r"/Users/dblyon/Downloads/results_orig_example1abundancecorr_without_filter_parents.tsv"
# fn_agotool_output = r"/Users/dblyon/Downloads/results_orig_example1abundancecorr_with_filter_parents.tsv"
# fn_agotool_output = r"/Users/dblyon/Downloads/results_orig_example1.tsv"
# fn_agotool_output = r"/Users/dblyon/Downloads/results_orig_example1_withoutFilterParents_localhost.tsv"
# fn_agotool_output = r"/Users/dblyon/Downloads/results_orig_example1_withFilterParents_localhost.tsv"
# fn_agotool_output = r"/Users/dblyon/Downloads/results_orig_example2.tsv"
# fn_agotool_output = r"/Users/dblyon/Downloads/results_orig_example3.tsv" 
# fn_agotool_output = r"/Users/dblyon/Downloads/results_orig_23144_1588787245.5236452.tsv" # manual restart uWSGI aquarius
fn_agotool_output = os.path.join(homedir, "modules/cpr/agotool/data/exampledata/Yeast_acetylation_abundance_corrected_example.tsv")
df = pd.read_csv(fn_agotool_output, sep="\t")
print(df.shape)

df = df[(df["FDR"] <= 0.05) & (df["FDR"] > 0)]
df["logFDR"] = df["FDR"].apply(lambda x: math.log(x)*-1)
df["FG_count_2_circle_size"] = scale_FG_count_2_plot_circle_size(df)
df["rank_2_transparency"] = df["rank"].apply(scale_rank_2_plot_transparency)
df["color"] = add_color_per_etype(df, palette_dict)

lineage_dict_enum = query.get_lineage_dict_enum(False, read_from_flat_files=True) # key: function enumeration, value: set of func enum array all parents
_, etype_2_edgesXYCoords_list_dict = find_edges_via_hierarchy(df, lineage_dict_enum, v1=False)

fn_example = os.path.join(homedir, "modules/cpr/agotool/data/exampledata/DF_Bokeh_example_for_plotting_playground.txt")
df.to_csv(fn_example, sep="\t", header=True, index=False)

!ls -lah {fn_agotool_output}
cond = df["FDR"] <= 0.05
sum(cond), len(cond)

graph_layout = get_graph_layout(df)
edge_renderer = get_edge_renderer(df, lineage_dict_enum)

(217, 20)
-rw-r--r--@ 1 davidl  IMB\grpvonMering   693K Jun 30 09:39 /Users/davidl/modules/cpr/agotool/data/exampledata/Yeast_acetylation_abundance_corrected_example.tsv


In [19]:
fn_example = os.path.join(homedir, "modules/cpr/agotool/data/exampledata/DF_Bokeh_example_for_plotting_playground.txt")
df = pd.read_csv(fn_example, sep="\t")

df["category"] = df["etype"].apply(lambda bubu: entityType_2_functionType_dict[bubu])
df_nodes = df
df_nodes["FG_count_2_circle_size"] = df_nodes["FG_count_2_circle_size"]*2
etype_2_funcNamePair_set_dict, df_coords = find_edges_via_hierarchy_v3(df_nodes, lineage_dict)

df_coords["category"] = df_coords["etype"].apply(lambda x: entityType_2_functionType_dict[x])
df_edges = df_coords

# create StackOverflow question

In [9]:
# fn_example = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/DF_Bokeh_example_for_plotting_playground.txt"
# fn_stackOverflow = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/StackOverflow_Bokeh_DF.txt"
# df = pd.read_csv(fn_example, sep="\t")
# x = df[df["category"].isin(['Gene Ontology cellular component', 'UniProt keywords'])] # 'Gene Ontology biological process',  'Gene Ontology molecular function', 
# x = x.groupby("etype").head(4).reset_index(drop=True)
# x = x[["etype", "category", "term", "description", "FG_count", "logFDR", "effectSize", "FG_count_2_circle_size", "rank_2_transparency", "color"]]
# x.to_csv(fn_stackOverflow, sep='\t', header=True, index=False)

# # x["category"] = x["etype"].apply(lambda bubu: etype_2_cat_dict[bubu])
# df_nodes = x.loc[0:6]
# df_nodes["FG_count_2_circle_size"] = df_nodes["FG_count_2_circle_size"]*2
# etype_2_funcNamePair_set_dict, df_coords = find_edges_via_hierarchy_v3(df_nodes, lineage_dict)

# entityType_2_functionType_dict = {-20: "Gene Ontology cellular component TEXTMINING",
#                               -21: "Gene Ontology biological process",
#                               -22: "Gene Ontology cellular component",
#                               -23: "Gene Ontology molecular function",
#                               -25: "Brenda Tissue Ontology",
#                               -26: "Disease Ontology",
#                               -51: "UniProt keywords",
#                               -52: "KEGG (Kyoto Encyclopedia of Genes and Genomes)",
#                               -53: "SMART (Simple Modular Architecture Research Tool)",
#                               -54: "INTERPRO",
#                               -55: "PFAM (Protein FAMilies)",
#                               -56: "PMID (PubMed IDentifier)",
#                               -57: "Reactome",
#                               -58: "WikiPathways"}

# df_coords["category"] = df_coords["etype"].apply(lambda x: entityType_2_functionType_dict[x])
# df_edges = df_coords
# df_nodes

# d = df_nodes.to_dict()
# # df_nodes_v2 = pd.DataFrame(d)
# # df_nodes == df_nodes_v2
# print(d)
# d = df_edges.to_dict()
# print(d)

# df_edges_v2 = pd.DataFrame(d)

### Self contained
# import pandas as pd
# from bokeh.plotting import figure, show, output_file, show, save
# from bokeh.models.sources import ColumnDataSource
# from bokeh.models import CustomJS, LabelSet, TapTool, HoverTool, BoxZoomTool, ResetTool, PanTool, SaveTool, Legend

# # settting up the data
# nodes = {'etype': {0: -22, 1: -22, 2: -22, 3: -22, 4: -51, 5: -51, 6: -51}, 'category': {0: 'Gene Ontology cellular component', 1: 'Gene Ontology cellular component', 2: 'Gene Ontology cellular component', 3: 'Gene Ontology cellular component', 4: 'UniProt keywords', 5: 'UniProt keywords', 6: 'UniProt keywords'}, 'term': {0: 'GO:0031981', 1: 'GO:0070013', 2: 'GO:0005634', 3: 'GO:0005654', 4: 'KW-0539', 5: 'KW-0805', 6: 'KW-0804'}, 'description': {0: 'nuclear lumen', 1: 'intracellular organelle lumen', 2: 'nucleus', 3: 'nucleoplasm', 4: 'Nucleus', 5: 'Transcription regulation', 6: 'Transcription'}, 'FG_count': {0: 303, 1: 411, 2: 612, 3: 138, 4: 487, 5: 141, 6: 153}, 'logFDR': {0: 22.318724310710312, 1: 18.092893813649173, 2: 15.775444841073195, 3: 9.373328781248528, 4: 22.527590362913926, 5: 17.179225147876295, 6: 14.291561909828324}, 'effectSize': {0: 0.12338222605694565, 1: 0.12769628990509058, 2: 0.13028472821397755, 3: 0.060396893874029335, 4: 0.14495254529767046, 5: 0.07420189818809317, 6: 0.07247627264883523}, 'FG_count_2_circle_size': {0: 12.0, 1: 15.711555169417897, 2: 23.395308427454392, 3: 12.0, 4: 18.616854908774982, 5: 12.0, 6: 12.0}, 'rank_2_transparency': {0: 1.0, 1: 0.9, 2: 0.8, 3: 0.7, 4: 1.0, 5: 0.9, 6: 0.8}, 'color': {0: '#2ca02c', 1: '#2ca02c', 2: '#2ca02c', 3: '#2ca02c', 4: '#9467bd', 5: '#9467bd', 6: '#9467bd'}}
# df_nodes = pd.DataFrame(nodes)
# edges = {'etype': {0: -51, 1: -22, 2: -22, 3: -22}, 'x_start': {0: 17.179225147876295, 1: 22.318724310710312, 2: 22.318724310710312, 3: 22.318724310710312}, 'x_stop': {0: 14.291561909828324, 1: 18.092893813649173, 2: 9.373328781248528, 3: 15.775444841073195}, 'y_start': {0: 0.07420189818809317, 1: 0.12338222605694565, 2: 0.12338222605694565, 3: 0.12338222605694565}, 'y_stop': {0: 0.07247627264883523, 1: 0.12769628990509058, 2: 0.060396893874029335, 3: 0.13028472821397755}, 'weight': {0: 1, 1: 1, 2: 1, 3: 1}, 'category': {0: 'UniProt keywords', 1: 'Gene Ontology cellular component', 2: 'Gene Ontology cellular component', 3: 'Gene Ontology cellular component'}}
# df_edges = pd.DataFrame(edges)

# output_file("Bokeh_plot.html")
# points = ColumnDataSource(data=df_nodes)
# # hover to show node label information
# hover = HoverTool(tooltips=[("term", "@term"), ("desc", "@description"), ("FG count", "@FG_count")])
# p = figure(plot_width=1000, plot_height=500, title = "", toolbar_sticky=False, toolbar_location='above',
#            tools=[hover, BoxZoomTool(), ResetTool(), PanTool(), SaveTool()]) 

# legend_it = []
# for category, group in df_nodes.groupby("category"):
#     group_edges = df_edges[df_edges["category"] == category]
#     for x_start, x_stop, y_start, y_stop, weight in zip(group_edges["x_start"], group_edges["x_stop"], group_edges["y_start"], group_edges["y_stop"], group_edges["weight"]):
#         plot_edges = p.line((x_start, x_stop), (y_start, y_stop), line_width=1, alpha=weight, color="grey")

#     color_of_etype = group.iloc[0].color
#     plot_ = p.circle(x="logFDR", y="effectSize", source=group, 
#                      size="FG_count_2_circle_size", fill_alpha="rank_2_transparency",                     
#                      color="color", muted_color="color", muted_alpha=0.2,
#                      nonselection_fill_alpha=0.2,
#                      nonselection_line_alpha=1.0,
#                      nonselection_fill_color=color_of_etype,
#                      nonselection_line_color=color_of_etype,
#                      selection_color=color_of_etype)
#     name = group.iloc[0].category    
#     legend_it.append((name, [plot_]))    
#     labels = ColumnDataSource(data=dict(x=[], y=[], t=[], ind=[]))
#     p.add_layout(LabelSet(x='x', y='y', text='t', y_offset=7, x_offset=7, source=labels, text_font_size="8pt"))

# legend = Legend(items=legend_it, location=(10, 0))
# legend.click_policy = "hide"
# p.add_layout(legend, 'right')
# plot_ = p.circle(x='logFDR', y='effectSize', source=points, size="FG_count_2_circle_size", alpha=0.0, color=None)
# save(p)

In [14]:
from bokeh.plotting import figure, show, output_file, show, save
from bokeh.models.sources import ColumnDataSource
from bokeh.models import CustomJS, LabelSet, TapTool, ZoomInTool, ZoomOutTool

output_file("Bokeh_plot_testing.html")
points = ColumnDataSource(data=df_nodes)
### Hover
hover = HoverTool(tooltips=[("term", "@term"), ("desc", "@description"), ("FG count", "@FG_count")])
p = figure(plot_width=1000, plot_height=500, title = "", toolbar_sticky=False, toolbar_location='above',
           tools=[hover, BoxZoomTool(), ResetTool(), PanTool(), SaveTool()]) 

legend_it = []
for category, group in df_nodes.groupby("category"):
    group_edges = df_edges[df_edges["category"] == category]
    for x_start, x_stop, y_start, y_stop, weight in zip(group_edges["x_start"], group_edges["x_stop"], group_edges["y_start"], group_edges["y_stop"], group_edges["weight"]):
        plot_edges = p.line((x_start, x_stop), (y_start, y_stop), line_width=1, alpha=weight, color="grey")

    color_of_etype = group.iloc[0].color
    plot_ = p.circle(x="logFDR", y="effectSize", source=group, 
                     size="FG_count_2_circle_size", fill_alpha="rank_2_transparency",                     
                     color="color", muted_color="color", muted_alpha=0.2,
                     nonselection_fill_alpha=0.2,
                     nonselection_line_alpha=1.0,
                     nonselection_fill_color=color_of_etype,
                     nonselection_line_color=color_of_etype,
                     selection_color=color_of_etype)
    name = group.iloc[0].category    
    legend_it.append((name, [plot_]))    
    labels = ColumnDataSource(data=dict(x=[], y=[], t=[], ind=[]))
    p.add_layout(LabelSet(x='x', y='y', text='t', y_offset=7, x_offset=7, source=labels, text_font_size="8pt"))

legend = Legend(items=legend_it, location=(10, 0))
legend.click_policy = "hide"
p.add_layout(legend, 'right')
plot_ = p.circle(x='logFDR', y='effectSize', source=points, size="FG_count_2_circle_size", alpha=0.0, color=None)

code_2_add_term_label_on_click = """
const data = labels.data
for (i=0; i<points.selected.indices.length; i++) {
    const ind = points.selected.indices[i]
    data.x.push(points.data.logFDR[ind])
    data.y.push(points.data.effectSize[ind])
    data.t.push(points.data.term[ind])
    data.ind.push(ind)
}
labels.change.emit()
"""
callback=CustomJS(args=dict(points=points, labels=labels), code=code_2_add_term_label_on_click)
p.add_tools(TapTool(callback=callback))
save(p)

'/Users/dblyon/modules/cpr/agotool/app/python/Bokeh_plot_testing.html'

In [ ]:
from bokeh.plotting import output_file, figure, show
from numpy.random import normal, uniform

meas_data_1 = normal(0, 1, 100)
meas_data_2 = uniform(-0.5, 0.5, 100)

output_file("myplot.html", title="My plot")
fig = figure(width=500, plot_height=500)

fig.line(x=range(0, len(meas_data_1)), y=meas_data_1)
fig.line(x=range(0, len(meas_data_2)), y=meas_data_2)

show(fig)

In [ ]:
# import numpy as np

# from bokeh.io import output_file, show
# from bokeh.layouts import row
# from bokeh.palettes import Viridis3
# from bokeh.plotting import figure
# from bokeh.models import CheckboxGroup, CustomJS

# output_file("line_on_off.html", title="line_on_off.py example")

# p = figure()
# props = dict(line_width=4, line_alpha=0.7)
# x = np.linspace(0, 4 * np.pi, 100)
# l0 = p.line(x, np.sin(x), color=Viridis3[0], legend="Line 0", **props)
# l1 = p.line(x, 4 * np.cos(x), color=Viridis3[1], legend="Line 1", **props)
# l2 = p.line(x, np.tan(x), color=Viridis3[2], legend="Line 2", **props)

# checkbox = CheckboxGroup(labels=["Line 0", "Line 1", "Line 2"],
#                          active=[0, 1, 2], width=100)
# checkbox.callback = CustomJS(args=dict(l0=l0, l1=l1, l2=l2, checkbox=checkbox),
#                              lang="coffeescript", code="""
# l0.visible = 0 in checkbox.active;
# l1.visible = 1 in checkbox.active;
# l2.visible = 2 in checkbox.active;
# """)

# layout = row(checkbox, p)
# show(layout)

# testing network

In [ ]:
# ToDo check that lineage dict does not go to root of blacklisted terms, since
# these could then connect all terms within an etype

# figure out which funcEnums should be connected
#   for each funcEnum get all parents, and intersect with all_current_funcEnums
# for each funcEnum if its not been visited yet:
#     get its x="logFDR", y="effectSize", and lineage:
#         add itself and each family member to x and y positions list 

In [ ]:
# df.index.tolist()

In [ ]:
fn_example = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/DF_Bokeh_example_for_plotting_playground.txt"
df = pd.read_csv(fn_example, sep="\t")
df.category.unique()

In [ ]:
from bokeh.plotting import figure, show, output_file, show, save
from bokeh.models.sources import ColumnDataSource
from bokeh.models import CustomJS, LabelSet, TapTool, ZoomInTool, ZoomOutTool

output_file("Bokeh_plot_testing.html")

fn_example = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/DF_Bokeh_example_for_plotting_playground.txt"
df = pd.read_csv(fn_example, sep="\t")
df = df[df["category"].isin(['Gene Ontology biological process', 'Gene Ontology cellular component', 'Gene Ontology molecular function', 'UniProt keywords'])]
df_nodes = df
# df = df.groupby("etype").head(10)
# etype_2_funcEnumPair_set_dict, etype_2_edgesXYCoords_list_dict = find_edges_via_hierarchy(df, lineage_dict_enum, v1=True)
etype_2_funcNamePair_set_dict, df_coords = find_edges_via_hierarchy_v3(df, lineage_dict)
df_coords["category"] = df_coords["etype"].apply(lambda x: entityType_2_functionType_dict[x])
df_edges = df_coords
points = ColumnDataSource(data=df_nodes)

### Hover
hover = HoverTool(tooltips=[("term", "@term"), ("desc", "@description"), ("FG count", "@FG_count")])
p = figure(plot_width=1000, plot_height=500, 
           title = "",
           toolbar_sticky=False,
           toolbar_location='above',
           tools=[hover, BoxZoomTool(), ZoomInTool(), ZoomOutTool(), ResetTool(), PanTool(), SaveTool()]) # , TapTool(), LassoSelectTool(), WheelZoomTool()
p.title.text_font_size = "1pt"
p.title.align = "center"
p.xaxis.axis_label = '-log(corrected p_value)'
p.xaxis.axis_label_text_font_style = "normal" # not "italic"
p.yaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label = 'effect size'
p.xgrid.grid_line_alpha = 0.5
p.ygrid.grid_line_alpha = 0.5
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None
line_width_edges = 1
legend_it = []
# for etype, group in df.groupby("etype"):
#     edgesXYCoords_list = etype_2_edgesXYCoords_list_dict.get(etype, [])
#     for edge in edgesXYCoords_list:
# #         if edge.weight > 0.7:
#         print(edge)
#         plot_edges = p.line(edge.x_vals, edge.y_vals, line_width=line_width_edges, alpha=edge.weight, color="grey")
for category, group in df_nodes.groupby("category"):
    group_edges = df_edges[df_edges["category"] == category]
    for x_start, x_stop, y_start, y_stop, weight in zip(group_edges["x_start"], group_edges["x_stop"], group_edges["y_start"], group_edges["y_stop"], group_edges["weight"]):
        plot_edges = p.line((x_start, x_stop), (y_start, y_stop), line_width=1, alpha=weight, color="grey")


    color_of_etype = group.iloc[0].color
    plot_ = p.circle(x="logFDR", y="effectSize", source=group, 
                     size="FG_count_2_circle_size", fill_alpha="rank_2_transparency",                     
                     color="color", muted_color="color", muted_alpha=0.2,
                     nonselection_fill_alpha=0.2,
                     nonselection_line_alpha=1.0,
                     nonselection_fill_color=color_of_etype,
                     nonselection_line_color=color_of_etype,
                     selection_color=color_of_etype)
    name = group.iloc[0].category    
    legend_it.append((name, [plot_]))    
    labels = ColumnDataSource(data=dict(x=[], y=[], t=[], ind=[]))
    p.add_layout(LabelSet(x='x', y='y', text='t', y_offset=7, x_offset=7, source=labels, text_font_size="8pt"))

legend = Legend(items=legend_it, location=(10, 0))
legend.click_policy = "hide" #"mute"
p.add_layout(legend, 'right')
plot_ = p.circle(x='logFDR', y='effectSize', source=points, size="FG_count_2_circle_size", alpha=0.0, color=None)

code_2_add_term_label_on_click = """
const data = labels.data
for (i=0; i<points.selected.indices.length; i++) {
    const ind = points.selected.indices[i]
    data.x.push(points.data.logFDR[ind])
    data.y.push(points.data.effectSize[ind])
    data.t.push(points.data.term[ind])
    data.ind.push(ind)
}
labels.change.emit()
"""

callback=CustomJS(args=dict(points=points, labels=labels), code=code_2_add_term_label_on_click)
p.add_tools(TapTool(callback=callback)) # , LassoSelectTool(callback=callback)

# show(p)
save(p)

In [ ]:
df_coords

In [ ]:
# output_notebook()
# output_file()
# comment out output_file(), run reset_output(), then run output_notebook() to keep from opening new tabs

# Save file
# save(p)
# Print out div and script
# script, div = components(p)
# print(div)
# print(script)

In [ ]:
import math

from bokeh.io import output_file, show
from bokeh.models import GraphRenderer, Oval, StaticLayoutProvider
from bokeh.palettes import Spectral8
from bokeh.plotting import figure

# N = 8
# node_indices = list(range(N))
etype = -21
graph_layout = get_graph_layout(df)
graph_layout = graph_layout[etype]

node_indices = list(graph_layout.keys())
edge_renderer = get_edge_renderer(df, lineage_dict_enum)
edge_renderer = edge_renderer[etype]

plot = figure(title='aGOtool is awesome', 
              x_range=(2,10), y_range=(-0.0,0.15),
              tools='', toolbar_location=None)

graph = GraphRenderer()

graph.node_renderer.data_source.add(node_indices, 'index')
# graph.node_renderer.data_source.add(Spectral8, 'color')
# graph.node_renderer.glyph = Circle()#height=0.1, width=0.2)#, fill_color='color')

graph.edge_renderer.data_source.data = edge_renderer # dict(start=[0]*N, end=node_indices)

### start of layout code
# circ = [i*2*math.pi/8 for i in node_indices]
# x = [math.cos(i) for i in circ]
# y = [math.sin(i) for i in circ]

# graph_layout = dict(zip(node_indices, zip(x, y)))
graph.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)

plot.renderers.append(graph)

# output_file('graph.html')
show(plot)

In [ ]:
N = 8
node_indices = list(range(N))
circ = [i*2*math.pi/8 for i in node_indices]
circ

In [ ]:
import networkx as nx

from bokeh.io import output_file, show
from bokeh.plotting import figure, from_networkx

G = nx.karate_club_graph()

plot = figure(title="Networkx Integration Demonstration", x_range=(-1.1,1.1), y_range=(-1.1,1.1),
              tools="", toolbar_location=None)

graph = from_networkx(G, nx.spring_layout, scale=2, center=(0,0))
plot.renderers.append(graph)

# output_file("networkx_graph.html")
show(plot)

In [ ]:
edge_renderer = dict(start=[0]*N, end=node_indices)

In [ ]:
edge_renderer

# etype_2_edgesXYCoords_list_dict

In [ ]:
# terms_2_check = ["GO:0006356", "GO:0006363", "GO:0006383", "GO:0006385"]
# df[df["etype"] == -22].sort_values("term", ascending=True)
# # df[df["term"].isin(terms_2_check)]

In [ ]:
# GO:0006385 is part of GO:0006383
# 5543 is part of 5540

In [ ]:
# funcEnum = 5529
# lineage_dict_enum[funcEnum]

In [ ]:
term = "GO:0006338"
df[df["term"] == term]

In [ ]:
len(edgesXYCoords_list)

In [ ]:
# edgesXYCoords_list = etype_2_edgesXYCoords_list_dict[-21]
# for edge in edgesXYCoords_list:
#     if edge.xvals
#     print(edge)

# ToDo

In [ ]:
# ToDo:
# on hovering over point highlight by changing color of surrounding circle e.g. to black
# on-click stick label --> toggle label
# reset should de-select highlight
# un-click to un-stick label
# select in plot show in table
# select in table show in plot
# selection should not change highlight --> can be undone by clicking on Legend
# Slider with customJS callback to select top_N terms (rank based)

# not working due to update:
# - Tap on a circle to add label

In [ ]:
# SIZE of circles depends on FG_count (number of proteins associated with given functional term)
# ALPHA (transparency) depends on rank within each functional category. Rank is analogous to s_value, a combination of p_value and effect size

# Best working example

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models.sources import ColumnDataSource
from bokeh.models import CustomJS, LabelSet, TapTool

fn_example = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/DF_Bokeh_example_for_plotting_playground.txt"
df = pd.read_csv(fn_example, sep="\t")
df = df[df["etype"] != -20]
points = ColumnDataSource(data=df)

### Hover
hover = HoverTool(tooltips=[("term", "@term"), ("desc", "@description"), ("FG count", "@FG_count")])
p = figure(plot_width=1000, plot_height=500, 
           title = "",
           toolbar_sticky=False,
           toolbar_location='above',
           tools=[hover, BoxZoomTool(), WheelZoomTool(), ResetTool(), PanTool(), SaveTool()]) # , TapTool(), LassoSelectTool()
p.title.text_font_size = "1pt"
p.title.align = "center"
p.xaxis.axis_label = '-log(corrected p_value)'
p.xaxis.axis_label_text_font_style = "normal" # not "italic"
p.yaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label = 'effect size'
p.xgrid.grid_line_alpha = 0.5
p.ygrid.grid_line_alpha = 0.5
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None

legend_it = []
for name, group in df.groupby("etype"):
    color_of_etype = group.iloc[0].color
    plot_ = p.circle(x="logFDR", y="effectSize", source=group, 
                     size="FG_count_2_circle_size", fill_alpha="rank_2_transparency",                     
                     color="color", muted_color="color", muted_alpha=0.2,
                     nonselection_fill_alpha=0.2,
                     nonselection_line_alpha=1.0,
                     nonselection_fill_color=color_of_etype,
                     nonselection_line_color=color_of_etype,
                     selection_color=color_of_etype)
    name = group.iloc[0].category
    legend_it.append((name, [plot_]))    
    labels = ColumnDataSource(data=dict(x=[], y=[], t=[], ind=[]))
    p.add_layout(LabelSet(x='x', y='y', text='t', y_offset=7, x_offset=7, source=labels, text_font_size="8pt"))

legend = Legend(items=legend_it, location=(10, 0))
legend.click_policy = "hide" #"mute"
p.add_layout(legend, 'right')
plot_ = p.circle(x='logFDR', y='effectSize', source=points, size="FG_count_2_circle_size", alpha=0.0, color=None)

code_2_add_term_label_on_click = """
const data = labels.data
for (i=0; i<points.selected.indices.length; i++) {
    const ind = points.selected.indices[i]
    data.x.push(points.data.logFDR[ind])
    data.y.push(points.data.effectSize[ind])
    data.t.push(points.data.term[ind])
    data.ind.push(ind)
}
labels.change.emit()
"""

callback=CustomJS(args=dict(points=points, labels=labels), code=code_2_add_term_label_on_click)
p.add_tools(TapTool(callback=callback)) # , LassoSelectTool(callback=callback)

show(p)

# compare KS Cython vs Scipy

In [ ]:
fn_scipy = r"~/Downloads/Example_1_Yeast_acetylation_abundance_correction_OUTPUT_SCIPY.txt"
fn_scipy = r"~/Downloads/Example_1_Yeast_acetylation_abundance_correction_OUTPUT_CYTHON.txt"
df = pd.read_csv(fn_scipy, sep='\t')

df, etype_2_edgesXYCoords_list_dict = get_df_ready_for_bokeh_plot(df)

from bokeh.plotting import figure, show
from bokeh.models.sources import ColumnDataSource
from bokeh.models import CustomJS, LabelSet, TapTool

points = ColumnDataSource(data=df)

### Hover
hover = HoverTool(tooltips=[("term", "@term"), ("desc", "@description"), ("FG count", "@FG_count")])
p = figure(plot_width=1000, plot_height=500, 
           title = "",
           toolbar_sticky=False,
           toolbar_location='above',
           tools=[hover, BoxZoomTool(), WheelZoomTool(), ResetTool(), PanTool(), SaveTool()]) # , TapTool(), LassoSelectTool()
p.title.text_font_size = "1pt"
p.title.align = "center"
p.xaxis.axis_label = '-log(corrected p_value)'
p.xaxis.axis_label_text_font_style = "normal" # not "italic"
p.yaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label = 'effect size'
p.xgrid.grid_line_alpha = 0.5
p.ygrid.grid_line_alpha = 0.5
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None

legend_it = []
for name, group in df.groupby("etype"):
    color_of_etype = group.iloc[0].color
    plot_ = p.circle(x="logFDR", y="effectSize", source=group, 
                     size="FG_count_2_circle_size", fill_alpha="rank_2_transparency",                     
                     color="color", muted_color="color", muted_alpha=0.2,
                     nonselection_fill_alpha=0.2,
                     nonselection_line_alpha=1.0,
                     nonselection_fill_color=color_of_etype,
                     nonselection_line_color=color_of_etype,
                     selection_color=color_of_etype)
    name = group.iloc[0].category
    legend_it.append((name, [plot_]))    
    labels = ColumnDataSource(data=dict(x=[], y=[], t=[], ind=[]))
    p.add_layout(LabelSet(x='x', y='y', text='t', y_offset=7, x_offset=7, source=labels, text_font_size="8pt"))

legend = Legend(items=legend_it, location=(10, 0))
legend.click_policy = "hide" #"mute"
p.add_layout(legend, 'right')
plot_ = p.circle(x='logFDR', y='effectSize', source=points, size="FG_count_2_circle_size", alpha=0.0, color=None)

code_2_add_term_label_on_click = """
const data = labels.data
for (i=0; i<points.selected.indices.length; i++) {
    const ind = points.selected.indices[i]
    data.x.push(points.data.logFDR[ind])
    data.y.push(points.data.effectSize[ind])
    data.t.push(points.data.term[ind])
    data.ind.push(ind)
}
labels.change.emit()
"""

callback=CustomJS(args=dict(points=points, labels=labels), code=code_2_add_term_label_on_click)
p.add_tools(TapTool(callback=callback)) # , LassoSelectTool(callback=callback)

show(p)

In [ ]:
df[df["over_under"] == "u"]

# New features

In [ ]:
fn_example = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/DF_Bokeh_example_for_plotting_playground.txt"
df = pd.read_csv(fn_example, sep="\t")
source = ColumnDataSource(data=df)
### Hover
hover = HoverTool(tooltips=[("term", "@term"), ("desc", "@description"), ("FG count", "@FG_count")])
p = figure(plot_width=1000, plot_height=500, 
           title="", 
           toolbar_sticky=False,
           toolbar_location='above',
           tools=[hover, BoxZoomTool(), LassoSelectTool(), ResetTool(), PanTool(), SaveTool(), TapTool(), ZoomIn(), ZoomOut()])

p.title.text_font_size = "1pt"
p.title.align = "center"
p.xaxis.axis_label = '-log(corrected p_value)'
p.xaxis.axis_label_text_font_style = "normal" # not "italic"
p.yaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label = 'effect size'
p.xgrid.grid_line_alpha = 0.5
p.ygrid.grid_line_alpha = 0.5
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None

legend_it = []
for name, group in df.groupby("etype"):
    color_of_etype = group.iloc[0].color
    plot_ = p.circle("logFDR", "effectSize", source=group,
                     size="FG_count_2_circle_size", fill_alpha="rank_2_transparency",                     
                     color="color", muted_color="color", muted_alpha=0.2,
                     selection_color=color_of_etype,
                     nonselection_fill_alpha=0.2,
                     nonselection_fill_color=color_of_etype,
                     nonselection_line_color=color_of_etype,
                     nonselection_line_alpha=1.0)
    renderer = p.text(x='logFDR', y='effectSize',
                      text='term', 
                      source=group)
    renderer.nonselection_glyph.text_alpha = 0.

    name = group.iloc[0].category
    legend_it.append((name, [plot_]))

legend = Legend(items=legend_it, location=(10, 0))
legend.click_policy = "mute"

p.add_layout(legend, 'right')
show(p)

In [ ]:
from bokeh.models import ColumnDataSource, Label, LabelSet, Range1d

source = ColumnDataSource(data=dict(height=[66, 71, 72, 68, 58, 62],
                                    weight=[165, 189, 220, 141, 260, 174],
                                    names=['Mark', 'Amir', 'Matt', 'Greg',
                                           'Owen', 'Juan']))

p = figure(title='Dist. of 10th Grade Students at Lee High',
           x_range=Range1d(140, 275))
p.scatter(x='weight', y='height', size=8, source=source)
p.xaxis[0].axis_label = 'Weight (lbs)'
p.yaxis[0].axis_label = 'Height (in)'

labels = LabelSet(x='weight', y='height', text='names', level='glyph',
              x_offset=5, y_offset=5, source=source, render_mode='canvas')

citation = Label(x=70, y=70, x_units='screen', y_units='screen',
                 text='Collected by Luke C. 2016-04-01', render_mode='css',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0)

from bokeh.models.callbacks import CustomJS

callback = CustomJS(code="""
// the event that triggered the callback is cb_obj:
// The event type determines the relevant attributes
console.log('Tap event occurred at x-position: ' + cb_obj.x)
""")

p = figure()
# execute a callback whenever the plot canvas is tapped
p.js_on_event('tap', callback)

p.add_layout(labels)
p.add_layout(citation)

show(p)

In [ ]:
from bokeh.models.callbacks import CustomJS

callback = CustomJS(code="""
// the event that triggered the callback is cb_obj:
// The event type determines the relevant attributes
console.log('Tap event occurred at x-position: ' + cb_obj.x)
""")

p = figure()
# execute a callback whenever the plot canvas is tapped
p.js_on_event('tap', callback)

In [ ]:
# define some points and a little graph between them
x = [2, 3, 5, 6, 8, 7]
y = [6, 4, 3, 8, 7, 5]
links = {0: [1, 2],
    1: [0, 3, 4],
    2: [0, 5],
    3: [1, 4],
    4: [1, 3],
    5: [2, 3, 4]}

p = figure(plot_width=400, plot_height=400, tools="", toolbar_location=None, title='Hover over points')

source = ColumnDataSource({'x0': [], 'y0': [], 'x1': [], 'y1': []})
sr = p.segment(x0='x0', y0='y0', x1='x1', y1='y1', color='olive', alpha=0.6, line_width=3, source=source, )
cr = p.circle(x, y, color='olive', size=30, alpha=0.4, hover_color='olive', hover_alpha=1.0)

# Add a hover tool, that sets the link data for a hovered circle
code = """
const links = %s
const data = {'x0': [], 'y0': [], 'x1': [], 'y1': []}
const indices = cb_data.index.indices
for (var i = 0; i < indices.length; i++) {
    const start = indices[i]
    for (var j = 0; j < links[start].length; j++) {
        const end = links[start][j]
        data['x0'].push(circle.data.x[start])
        data['y0'].push(circle.data.y[start])
        data['x1'].push(circle.data.x[end])
        data['y1'].push(circle.data.y[end])
    }
}
segment.data = data
""" % links

# callback = CustomJS(args={'circle': cr.data_source, 'segment': sr.data_source}, code=code)
# p.add_tools(HoverTool(tooltips=None, callback=callback, renderers=[cr]))
p.add_tools(HoverTool(tooltips=None, renderers=[cr]))

show(p)

In [ ]:
# https://discourse.bokeh.org/t/how-do-i-do-pre-selection-on-rows-in-a-bokeh-datatable-and-display-such-selection/2326

from bokeh.plotting import figure, curdoc
from bokeh.models import CustomJS
from bokeh.models.sources import ColumnDataSource
from bokeh.layouts import row, column
from bokeh.models.widgets import DataTable, TableColumn, Button
import random

data = dict(index=list(range(10)), x = list(range(10)), y = list(range(10)), z = ["some other data"] * 10)
# source1 = ColumnDataSource(data)
source2 = ColumnDataSource(dict(computed_indices=[1,2,3]))
fig1 = figure(plot_width=300, plot_height=300)
fig1.circle(x="x", y="y", size=10, source=source1)
columns = [
TableColumn(field="y", title="Y"),
TableColumn(field="z", title="Text"),
]

data_table = DataTable(source=source1, columns=columns, width=400, height=280)
button = Button(label="Select")
button.callback = CustomJS(args=dict(source1=source1, source2=source2), code="""
source1[‘selected’][‘1d’].indices = source2[‘data’][‘computed_indices’]
source1.properties.selected.change.emit() //DataTable highlights when the selected property “change” signal is emitted
source1.select.emit() //GlyphRenderer (the circles in this example) re-render when a data source’s “select” signal is emitted
""")

def compute_indices():
    computed_indices = random.sample(list(range(10)), 3)
    new_data = dict(computed_indices=computed_indices)
    source2.data = new_data
    button2 = Button(label="Compute indices")
    button2.on_click(compute_indices)
    curdoc().add_root(row(fig1, data_table, column(button2, button)))

# Latest

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models.sources import ColumnDataSource
from bokeh.models import CustomJS, LabelSet, TapTool
from bokeh.models import ColumnDataSource, CustomJS, Slider

fn_example = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/DF_Bokeh_example_for_plotting_playground.txt"
df = pd.read_csv(fn_example, sep="\t")
points = ColumnDataSource(data=df)

hover = HoverTool(tooltips=[("term", "@term"), ("desc", "@description"), ("FG count", "@FG_count")])
p = figure(plot_width=1000, plot_height=500, title = " ", toolbar_sticky=False, toolbar_location='above',
           tools=[hover, BoxZoomTool(), WheelZoomTool(), PanTool(), SaveTool(), LassoSelectTool()]) #, TapTool(), (), ResetTool()
p.title.text_font_size = "1pt"
p.title.align = "center"
p.xaxis.axis_label = '-log(corrected p_value)'
p.xaxis.axis_label_text_font_style = "normal" # not "italic"
p.yaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label = 'effect size'
p.xgrid.grid_line_alpha = 0.5
p.ygrid.grid_line_alpha = 0.5
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None

legend_it = []
for name, group in df.groupby("etype"):
    color_of_etype = group.iloc[0].color
    plot_ = p.circle(x="logFDR", y="effectSize", source=group, 
                     size="FG_count_2_circle_size", fill_alpha="rank_2_transparency", color="color", 
                     muted_color="color", muted_alpha=0.2, nonselection_fill_alpha=0.2,
                     nonselection_line_alpha=1.0, nonselection_fill_color=color_of_etype,
                     nonselection_line_color=color_of_etype)#, selection_color=color_of_etype)
    name = group.iloc[0].category
    legend_it.append((name, [plot_]))
    
legend = Legend(items=legend_it, location=(10, 0))
legend.click_policy = "mute"
p.add_layout(legend, 'right')

plot_ = p.circle(x='logFDR', y='effectSize', source=points, size="FG_count_2_circle_size", alpha=0.0, color=None)
labels = ColumnDataSource(data=dict(x=[], y=[], t=[], ind=[]))
p.add_layout(LabelSet(x='x', y='y', text='t', y_offset=7, x_offset=7, source=labels, text_font_size="8pt"))

callback_2_add_term_label = CustomJS(args=dict(points=points, labels=labels), code="""
const data = labels.data;
for (i=0; i<points.selected.indices.length; i++) {
    const ind = points.selected.indices[i]
    data.x.push(points.data.logFDR[ind])
    data.y.push(points.data.effectSize[ind])
    data.t.push(points.data.term[ind])
    data.ind.push(ind)
};
labels.change.emit();""")

p.add_tools(TapTool(callback=callback_2_add_term_label))#, LassoSelectTool(callback=callback_2_add_term_label))
show(p)

In [ ]:
# Best working example adapted

from bokeh.plotting import figure, show
from bokeh.models.sources import ColumnDataSource
from bokeh.models import CustomJS, LabelSet, TapTool

fn_example = r"/Users/dblyon/modules/cpr/agotool/data/exampledata/DF_Bokeh_example_for_plotting_playground.txt"
df = pd.read_csv(fn_example, sep="\t")
points = ColumnDataSource(data=df)

### Hover
hover = HoverTool(tooltips=[("term", "@term"), ("desc", "@description"), ("FG count", "@FG_count")])
p = figure(plot_width=1000, plot_height=500, 
           title = "",
           toolbar_sticky=False,
           toolbar_location='above',
           tools=[hover, BoxZoomTool(), WheelZoomTool(), ResetTool(), PanTool(), SaveTool()]) # , TapTool(), LassoSelectTool()
p.title.text_font_size = "1pt"
p.title.align = "center"
p.xaxis.axis_label = '-log(corrected p_value)'
p.xaxis.axis_label_text_font_style = "normal" # not "italic"
p.yaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label = 'effect size'
p.xgrid.grid_line_alpha = 0.5
p.ygrid.grid_line_alpha = 0.5
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None

legend_it = []
for name, group in df.groupby("etype"):
    color_of_etype = group.iloc[0].color
    plot_ = p.circle(x="logFDR", y="effectSize", source=group, 
                     size="FG_count_2_circle_size", fill_alpha="rank_2_transparency",                     
                     color="color", muted_color="color", muted_alpha=0.2,
                     nonselection_fill_alpha=0.2,
                     nonselection_line_alpha=1.0,
                     nonselection_fill_color=color_of_etype,
                     nonselection_line_color=color_of_etype,
                     selection_color=color_of_etype)
    name = group.iloc[0].category
    legend_it.append((name, [plot_]))    
    labels = ColumnDataSource(data=dict(x=[], y=[], t=[], ind=[]))
    p.add_layout(LabelSet(x='x', y='y', text='t', y_offset=7, x_offset=7, source=labels, text_font_size="8pt"))

legend = Legend(items=legend_it, location=(10, 0))
legend.click_policy = "mute"
p.add_layout(legend, 'right')
plot_ = p.circle(x='logFDR', y='effectSize', source=points, size="FG_count_2_circle_size", alpha=0.0, color=None)

callback_add_term_label = CustomJS(args=dict(points=points, labels=labels), 
                                   code="""const data = labels.data;
for (i=0; i<points.selected.indices.length; i++) {
    const ind = points.selected.indices[i];
    data.x.push(points.data.logFDR[ind]);
    data.y.push(points.data.effectSize[ind]);
    data.t.push(points.data.term[ind]);
    data.ind.push(ind);
};
labels.change.emit();""")

# callback=CustomJS(args=dict(points=points, labels=labels), code=code_2_add_term_label_on_click)
p.add_tools(TapTool(callback=callback_add_term_label)) # , LassoSelectTool(callback=callback)

show(p)


p.js_on_event(events.SelectionGeometry, CustomJS(args=dict(div=div), code="""
div.text = "Selection! <p> <p>" + JSON.stringify(cb_obj.geometry, undefined, 2);
"""))

In [ ]:
callback = CustomJS(args=dict(s1=s1, s2=s2), code="""
    var inds = s1.selected.indices;
    if (inds.length == 0)
        return;

    var ym = 0
    for (var i = 0; i < inds.length; i++) {
        ym += s1.data.y[inds[i]]
    }
    
    ym /= inds.length
    s2.data.ym = [ym, ym]

    // necessary becasue we mutated source.data in-place
    s2.change.emit();  
""")

s1.selected.js_on_change('indices', callback)

In [ ]:
labels.__dict__

In [ ]:
labels.selected.__dict__

In [ ]:
# Add a hover tool, that sets the link data for a hovered circle
code = """
const links = %s
const data = {'x0': [], 'y0': [], 'x1': [], 'y1': []}
const indices = cb_data.index.indices
for (var i = 0; i < indices.length; i++) {
    const start = indices[i]
    for (var j = 0; j < links[start].length; j++) {
        const end = links[start][j]
        data['x0'].push(circle.data.x[start])
        data['y0'].push(circle.data.y[start])
        data['x1'].push(circle.data.x[end])
        data['y1'].push(circle.data.y[end])
    }
}
segment.data = data
""" % links

In [ ]:
from datetime import date
from random import randint

from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, DataTable, DateFormatter, TableColumn


data = dict(
        dates=[date(2014, 3, i+1) for i in range(10)],
        downloads=[randint(0, 100) for i in range(10)],
    )
source = ColumnDataSource(data)

columns = [
        TableColumn(field="dates", title="Date", formatter=DateFormatter()),
        TableColumn(field="downloads", title="Downloads"),
    ]
data_table = DataTable(source=source, columns=columns, width=400, height=280)

show(data_table)
